In [127]:
import math
from scipy.optimize import minimize

In [128]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
size = 10  #how many rows from the original data?
card = 5   #cardinality
sigma = np.genfromtxt("orl200-005-a.mat", skip_header=1, dtype='int')
sigma = sigma[0:size,0:size]
mu = pd.read_csv("orl200-005-a.txt",sep=" ", index_col=False, skiprows=1,names=['mu_i', 'ignore'])
rho = np.genfromtxt("orl200-005-a.rho")
rho = rho.item()
mu = mu.iloc[0:size,0]
mu = np.asarray(mu)
granice = pd.read_csv("orl200-005-a.bds", sep=" ", index_col=False,names=['l_i', 'u_i'])
granice = granice.transpose()
u_i = granice.iloc[1,0:size]
u_i = np.asarray(u_i)

$u_1,\ldots,u_8$ are Lagrangian multipliers 

$u_4,\ldots,u_9$ are arrays of size $size$

$\mathbf x = (x_1,\ldots,x_n,y_1,\ldots,y_n)$ 

In [129]:
def objective(x):
    obj = x[0:size].dot(sigma).dot(x[0:size])-(1.0/(k*4))*np.square(u1)-(1.0/(k*4))*np.square(u2)-\
    (1.0/(k*4))*np.square(u3)-(1.0/(k*4))*np.square(u4).sum()-(1.0/(k*4))*np.square(u5).sum()\
    -(1.0/(k*4))*np.square(u6).sum()-(1.0/(k*4))*np.square(u7).sum()-(1.0/(k*4))*np.square(u8).sum()-\
    (1.0/(k*4))*np.square(u9).sum()
    obj += (1.0*k)*max(0, x[0:size].sum() - 1 + (u1/k)/2)**2
    obj += (1.0*k)*max(0, rho-mu.dot(x[0:size])+(u2/k)/2)**2
    obj += (1.0*k)*max(0, size-card-x[size:size*2].sum()+(u3/k)/2)**2
    #constraints for xs
    for i in range(size):
        obj += (1.0*k)*max(0, x[i]-u_i[i]+(u4[i]/k)/2)**2
        obj += (1.0*k)*max(0, -x[i]+(u5[i]/k)/2)**2
        obj += (1.0*k)*max(0, -x[i+size]+(u6[i]/k)/2)**2  #y
        obj += (1.0*k)*max(0, x[i+size]-1+(u7[i]/k)/2)**2
        if(x[i]+x[i+size]>=2*t):
            obj += (1.0*k)*max(0, (x[i]-t)*(x[i+size]-t)+(u8[i]/k)/2)**2
        else:
            obj += (1.0*k)*max(0, -1/2*((x[i]-t)**2+(x[i+size]-t)**2)+(u8[i]/k)/2)**2
        if(-x[i]+x[i+size]>=2*t):
            obj += (1.0*k)*max(0, (-x[i]-t)*(x[i+size]-t)+(u9[i]/k)/2)**2
        else:
            obj += (1.0*k)*max(0, -1/2*((-x[i]-t)**2+(x[i+size]-t)**2)+(u9[i]/k)/2)**2
    return obj

In [130]:
def phi(a, b, t):
    if(a+b>=2*t):
        result = (a-t)*(b-t)
    else:
        result = -1/2*((a-t)**2+(b-t)**2)
    return result

def phi_(a, b, t):
    if(-a+b >= 2*t):
        result = (-a-t)*(b-t)
    else:
        result = -1/2*((-a-t)**2+(b-t)**2)
    return result

In [131]:
%%time
t = 1
k = 1/t   #penalty parameter
u1 = 0
u2 = 0
u3 = 0
u4 = np.zeros(size)
u5 = np.zeros(size)
u6 = np.zeros(size)
u7 = np.zeros(size)
u8 = np.zeros(size)
u9 = np.zeros(size)
viol = math.inf     #at the beggining violence is infinity
x0 = np.append(np.zeros(size), np.ones(size))  #starting point
old_viol = viol
eps = 10**-7 #tolerance level
cond = 1
step = 0.7
it = 0
while ((eps <= viol) & (10**-10 <= t)):
    it += 1
    #inner loop
    x = minimize(objective, x0).x
    violence = np.zeros(3+5*size)
    violence[0] = max(0,x[0:size].sum()-1)
    violence[1] = max(0,size-card-x[size:size*2].sum())
    violence[2] = max(0,rho-mu.dot(x[0:size]))
    for i in range(size):
        violence[i+3] = max(0,phi(x[i],x[i+size],t))
        violence[(i+3)+size] = max(0, -x[i+size])
        violence[(i+3)+2*size] = max(0, x[i+size]-1)
        violence[(i+3)+3*size] = max(0, -x[i])
        violence[(i+3)+4*size] = max(0, x[i]-u_i[i])
    viol = max(violence) 
    if (viol <= (1.0/4)*old_viol):
        u1 = u1 + max(-u1, (2*k)*(x.sum()-1))
        u2 = u2 + max(-u2, (2*k)*(rho-mu.dot(x[0:size])))
        u3 = u3 + max(-u3, (2*k)*(size-card-x[size:size*2].sum()))
        for i in range(size):
            u4[i] = u4[i] + max(-u4[i], (2*k)*(x[i]-u_i[i]))
            u5[i] = u5[i] + max(-u5[i], (2*k)*(-x[i]))
            u6[i] = u6[i] + max(-u6[i], (2*k)*(-x[i+size]))
            u7[i] = u7[i] + max(-u7[i], (2*k)*(x[i+size]-1))
            u8[i] = u8[i] + max(-u8[i], (2*k)*phi(x[i], x[i+size], t))
            u9[i] = u9[i] + max(-u9[i], (2*k)*phi_(x[i], x[i+size], t))
    if (viol > (1.0/4)*old_viol):
        t = t*step
        k = (k*(1/step))
    print('{} -> {}'.format(it, x[0:size].dot(sigma).dot(x[0:size])))
    x0 = x
    old_viol = viol

1 -> 3.912326193259069e-12
2 -> 0.6318007006163723
3 -> 0.5606174841911202
4 -> 0.46732476841001086
5 -> 0.35053102554764015
6 -> 0.2154991859310379
7 -> 0.08258618208062182
8 -> 0.002105252313391609
9 -> 1.1717370395784237e-09
10 -> 2.3094054966636592e-09
11 -> 4.5976943057841045e-09
12 -> 9.218810844602016e-09
13 -> 1.858107148129073e-08
14 -> 3.758968382901756e-08
15 -> 7.6241196382289e-08
16 -> 1.549179022831105e-07
17 -> 3.161168703762089e-07
18 -> 6.440161894059224e-07
19 -> 1.3078737208344104e-06
20 -> 2.6660102590103885e-06
21 -> 5.435865689638351e-06
22 -> 1.1085000320934614e-05
23 -> 2.2605783269129906e-05
24 -> 4.6097687800205024e-05
25 -> 9.398754717347883e-05
26 -> 0.00018216581063877304
27 -> 0.00034530866656992497
28 -> 0.0006283560471424674
29 -> 0.0012046186659657144
30 -> 0.0024078784789224998
31 -> 0.004864509790477422
32 -> 0.009843691123150181
33 -> 0.019892790441680238
34 -> 0.04003861690387736
35 -> 0.0801406994762709
36 -> 0.1591374291224184
37 -> 0.312511039576

In [110]:
x[0:size].dot(sigma).dot(x[0:size])

71.99746776293883

In [104]:
x[0:size].sum()

0.9858517138167737

In [124]:
viol

1.3682054097031524

In [125]:
t

1.0000000000000002e-12

In [ ]:
t